<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Monday 30 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 1: Staging data for analysis</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to process the separate netCDF4 files for the model drivers (X<sub>i=1, 2, ... M</sub>) and model target (Y) into a unified tabular data frame, exported as a compressed comma separated value (CSV) file. This file is subsequently used in the Bayesian inference study that forms the second notebook in this experiment. The advantage of the pre-processing the data separately to the analysis allows for it be quickly staged on demand. Of course other file formats may be more advantageous for greater compression (e.g. SQLite3 database file).
</p>
<br>
<b>You will need to run this notebook to prepare the dataest before you attempt the Bayesian analysis in Part 2</b>.
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
<hr>
</font>
</center>
</div>

## Load libraries

In [8]:
# data munging and analytical libraries 
import re
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset 

# graphical libraries
import matplotlib.pyplot as plt
%matplotlib inline

# set paths
outPath = "../data/vegFracControls.csv"

## Import and clean data

Set the directory path and look for all netcdf files that correspond to the model drivers and target.

In [9]:
driver_paths = [os.path.join(dp, f) for (dp, _, fn) in os.walk("../data/driving_Data/") for f in fn if f.endswith('.nc')]

driver_names = [re.search('^[a-zA-Z_]*', os.path.basename(fp)).group(0) for fp in driver_paths]

file_table = pd.DataFrame({'filepath': driver_paths, 'file_name': driver_names})
file_table

,file_name,filepath
0,BurntArea,../data/driving_Data/BurntArea.nc
1,crop,../data/driving_Data/crop.nc
2,MADD_CMORPH,../data/driving_Data/MADD_CMORPH.nc
3,MADD_CRU,../data/driving_Data/MADD_CRU.nc
4,MADD_GPCC,../data/driving_Data/MADD_GPCC.nc
5,MADD_MSWEP,../data/driving_Data/MADD_MSWEP.nc
6,MADM_CMORPH,../data/driving_Data/MADM_CMORPH.nc
7,MADM_CRU,../data/driving_Data/MADM_CRU.nc
8,MADM_GPCC,../data/driving_Data/MADM_GPCC.nc
9,MADM_MSWEP,../data/driving_Data/MADM_MSWEP.nc


Define a function to extract the variable values from each netCDF4 file. Variables are flattened from a 3 dimensional array to 1 dimensional version, pooling all values both spatially and temporily. 

Don't know if this is the correct way to do this, but will come back to it once I understand the model (and its optimisation) better.

In [10]:
from pdb import set_trace as browser
def nc_extract(fpath):
    print("Processing: {0}".format(fpath))
    with Dataset(fpath, 'r') as nc_file:
        gdata = np.array(nc_file.variables['layer'][:,:])
        gflat = gdata.flatten()
        #browser()
        if type(gdata) == np.ma.core.MaskedArray:
            return gflat[~gflat.mask].data
        else:
            return gflat.data

Execute the above function on all netCDF4 file paths.

In [11]:
values = [nc_extract(dp) for dp in driver_paths]
for i in values: print(i.shape)

Processing: ../data/driving_Data/BurntArea.nc
Processing: ../data/driving_Data/crop.nc
Processing: ../data/driving_Data/MADD_CMORPH.nc
Processing: ../data/driving_Data/MADD_CRU.nc
Processing: ../data/driving_Data/MADD_GPCC.nc
Processing: ../data/driving_Data/MADD_MSWEP.nc
Processing: ../data/driving_Data/MADM_CMORPH.nc
Processing: ../data/driving_Data/MADM_CRU.nc
Processing: ../data/driving_Data/MADM_GPCC.nc
Processing: ../data/driving_Data/MADM_MSWEP.nc
Processing: ../data/driving_Data/MAP_CMORPH.nc
Processing: ../data/driving_Data/MAP_CRU.nc
Processing: ../data/driving_Data/MAP_GPCC.nc
Processing: ../data/driving_Data/MAP_MSWEP.nc
Processing: ../data/driving_Data/MAT.nc
Processing: ../data/driving_Data/MConc_CMORPH.nc
Processing: ../data/driving_Data/MConc_CRU.nc
Processing: ../data/driving_Data/MConc_GPCC.nc
Processing: ../data/driving_Data/MConc_MSWEP.nc
Processing: ../data/driving_Data/MDDM_CMORPH.nc
Processing: ../data/driving_Data/MDDM_CRU.nc
Processing: ../data/driving_Data/MDD

Turn this into a dataframe for the analysis.

In [12]:
# turn list into a dataframe
fire_df = pd.DataFrame(np.array(values).T, columns=driver_names)

# replace null flags with pandas null
fire_df.replace(fire_df["BurntArea"][0], np.nan, inplace=True)

# drop all null rows (are ocean and not needed in optim)
fire_df.dropna(inplace=True)

Check that we've built it correctly.

In [13]:
fire_df.head()


,BurntArea,crop,MADD_CMORPH,MADD_CRU,MADD_GPCC,MADD_MSWEP,MADM_CMORPH,MADM_CRU,MADM_GPCC,MADM_MSWEP,...,MDDM_CRU,MDDM_GPCC,MDDM_MSWEP,MTWM,pas,PopDen,SW,SW,TreeCover,urban
108,0.000103,0.000000,0.937685,1.0,0.954822,0.838237,0.906849,1.0,0.997232,0.910575,...,1.0,1.0,0.975487,23.650000,0.004338,1.892931,202.611023,153.271469,0.023839,0.0
109,0.000038,0.000007,0.949989,1.0,0.965384,0.876759,0.988337,1.0,0.998174,0.976160,...,1.0,1.0,0.997696,24.842857,0.000955,2.009711,210.107697,145.774796,0.011756,0.0
110,0.000006,0.000002,0.907838,1.0,0.954631,0.866844,0.901959,1.0,0.998457,0.953528,...,1.0,1.0,0.997677,26.021429,0.001094,1.894917,211.678619,144.203873,0.004554,0.0
111,0.000008,0.000014,0.857775,1.0,0.949744,0.874905,0.729050,1.0,0.997271,0.966560,...,1.0,1.0,0.994739,27.778572,0.011957,0.497301,214.360779,141.521713,0.005536,0.0
112,0.000000,0.000000,0.892677,1.0,0.939389,0.894754,0.812315,1.0,0.994605,0.959878,...,1.0,1.0,0.998233,28.535715,0.014711,0.180588,208.746124,147.136368,0.019554,0.0


Export this to disk to be used by the analysis notebook - used gzip compression to save on space. Beware, because of there are approximation 10 million rows of data, this may take some time.

In [14]:
savepath = os.path.expanduser(outPath)
fire_df.to_csv(savepath, index=False)

<div>
<br>
<br>
<br>
<center>
<font size="5">
<a style="font-weight: bold; size: 5" href="http://localhost:8888/notebooks/notebooks/bayesian_inference.ipynb">Part 2: click here</a>
</font>
</center>
</div>